In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import time
import random
import pdb


### Make a list of the urls of the categories in the forum

In [2]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news',
 'https://curltalk.naturallycurly.com/categories/feedback',
 'https://curltalk.naturallycurly.com/categories/c2',
 'https://curltalk.naturallycurly.com/categories/c3',
 'https://curltalk.naturallycurly.com/categories/c4',
 'https://curltalk.naturallycurly.com/categories/general-discussion-about-curly-hair',
 'https://curltalk.naturallycurly.com/categories/salon-stylist-chatter',
 'https://curltalk.naturallycurly.com/categories/transitioning',
 'https://curltalk.naturallycurly.com/categories/thin-thinning-hair',
 'https://curltalk.naturallycurly.com/categories/readers-product-reviews',
 'https://curltalk.naturallycurly.com/categories/hall-of-shame-discussion',
 'https://curltalk.naturallycurly.com/categories/hall-of-fame-discussion'

#### Get the discussion topic urls for the first x pages of the category

In [24]:
# %%time

link_listdiscussion = []
for i in range(0, 100):
    url = f'{categories[51]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    #time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
#     time.sleep(random.randint(1,5))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/politics/p0
https://curltalk.naturallycurly.com/categories/politics/p1
https://curltalk.naturallycurly.com/categories/politics/p2
https://curltalk.naturallycurly.com/categories/politics/p3
https://curltalk.naturallycurly.com/categories/politics/p4
https://curltalk.naturallycurly.com/categories/politics/p5
https://curltalk.naturallycurly.com/categories/politics/p6
https://curltalk.naturallycurly.com/categories/politics/p7
https://curltalk.naturallycurly.com/categories/politics/p8
https://curltalk.naturallycurly.com/categories/politics/p9
https://curltalk.naturallycurly.com/categories/politics/p10
https://curltalk.naturallycurly.com/categories/politics/p11
https://curltalk.naturallycurly.com/categories/politics/p12
https://curltalk.naturallycurly.com/categories/politics/p13
https://curltalk.naturallycurly.com/categories/politics/p14
https://curltalk.naturallycurly.com/categories/politics/p15
https://curltalk.naturallycurly.com/categories/pol

https://curltalk.naturallycurly.com/categories/politics/p91
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p92
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p93
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p94
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p95
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p96
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p97
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p98
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/politics/p99
Error improper response code. Code is 404
711
Error improper response code. Code is 404
1
Error improper response code. Code

ConnectionError: ('Connection aborted.', OSError("(50, 'ENETDOWN')"))

In [18]:
len(list_for_mongo)

1999

### Create a mongoDB and have data stored from scrape 

In [19]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [20]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [21]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

500490


In [22]:
test_set = set(test)

In [23]:
# Number of unique entries so far
len(test_set)
#7950

7950